## Load in data

In [94]:
%load_ext autoreload
%autoreload 2
from ais_dataloader import *
from gp_kernel_ship_classification_dataset import *
from gp_kernel_ship_classification_trainer import *
from gp_kernel_ship_classification_network import *

import ipywidgets as widgets
from IPython.display import display
from plotting_utils import *



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using device: cpu


In [49]:
date_range = pd.date_range(start='2024-01-01', end='2024-01-01', freq='D')
gp_regression_dataset = AISTrajectoryRegressionDataset(date_range, device)


Loading cached dataframe from data/processed/processed_AIS_df_2024_01_01_2024_01_01.pkl


Scaling trajectories for each MMSI: 100%|██████████| 3453/3453 [00:01<00:00, 3213.55it/s]



===== Dataset Statistics =====
Total number of AIS messages: 2128288
Number of unique MMSIs: 3453
Date range: 2024-01-01 00:00:00 to 2024-01-01 23:59:59


## Fit GP Models

In [ ]:
import torch
import gpytorch
from multioutput_gp import *

num_trajectories = 3453
models = {}
likelihoods = {}
losses = {}

for idx in range(num_trajectories):
    # mmsi, times, state_trajectory = gp_regression_dataset[idx]
    mmsi, times, state_trajectory = gp_regression_dataset[idx]
    
    print(f"\nFitting GP for trajectory {idx+1}/{num_trajectories} for MMSI {mmsi}")
    
    X = times.detach().unsqueeze(1).to(device)
    Y = state_trajectory.detach().to(device)
    
    num_outputs = Y.shape[1]
    
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=num_outputs, noise_prior=gpytorch.priors.NormalPrior(loc=0.25, scale=0.25)).to(device)
    model = MultiOutputExactGPModel(X, Y, likelihood, num_outputs=num_outputs).to(device)

    with gpytorch.settings.cholesky_jitter(1e-3):
        loss, model, likelihood = train_model(model, likelihood, X, Y, num_epochs=100, lr=0.1, mmsi=mmsi)

    print(f"Loss: {loss.item()}")
    # models.append(model)
    # likelihoods.append(likelihood)
    # losses.append(loss.item())
    models[mmsi] = model
    likelihoods[mmsi] = likelihood
    losses[mmsi] = loss.item()


Fitting GP for trajectory 1/100 for MMSI 3660489


GP Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniconda/base/envs/gp/lib/python3.13/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-03 to the diagonal
  warnings.warn(
GP Training Progress: 100%|██████████| 100/100 [00:00<00:00, 187.32it/s]


Loss: 1.1078234910964966

Fitting GP for trajectory 2/100 for MMSI 203661016


GP Training Progress: 100%|██████████| 100/100 [00:04<00:00, 22.71it/s]


Loss: 1.0687633752822876

Fitting GP for trajectory 3/100 for MMSI 205691000


...

Loss: 0.7765158414840698

Fitting GP for trajectory 99/100 for MMSI 229894000


GP Training Progress: 100%|██████████| 100/100 [00:00<00:00, 275.40it/s]


Loss: 1.3274036645889282

Fitting GP for trajectory 100/100 for MMSI 229903000


GP Training Progress: 100%|██████████| 100/100 [00:12<00:00,  7.81it/s]

Loss: 0.8263754844665527


In [63]:
model = next(iter(models.values()))  # Get one of the models for demonstration
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:42} value = {param.tolist()}')
    
print()
print(model.covar_module.data_covar_module.kernels[0].lengthscale.item())

Parameter name: likelihood.raw_task_noises                 value = [-0.6876325607299805, -0.581544041633606, -0.9424332976341248, -0.5385560393333435, -0.9444971680641174, -0.9455525875091553]
Parameter name: likelihood.raw_noise                       value = [-0.9224559664726257]
Parameter name: mean_module.base_means.0.raw_constant      value = 0.06738271564245224
Parameter name: mean_module.base_means.1.raw_constant      value = -0.10569565743207932
Parameter name: mean_module.base_means.2.raw_constant      value = -4.691566209658049e-05
Parameter name: mean_module.base_means.3.raw_constant      value = 0.0004817449953407049
Parameter name: mean_module.base_means.4.raw_constant      value = -0.0001320840819971636
Parameter name: mean_module.base_means.5.raw_constant      value = -0.0004329355724621564
Parameter name: covar_module.task_covar_module.covar_factor value = [[-0.3279779851436615], [-0.39416366815567017], [-0.001594276400282979], [-0.0971570685505867], [0.01094231568276882

In [64]:
# Set the model and likelihood to evaluation mode
model.eval()
likelihood.eval()

# Generate test inputs (e.g., evenly spaced time points)
test_times = torch.linspace(times.min(), times.max(), 200).unsqueeze(1).to(device)

# Make predictions
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cholesky_jitter(1e-3):
    predictions = likelihood(model(test_times))
    mean = predictions.mean
    lower, upper = predictions.confidence_region()



/opt/homebrew/Caskroom/miniconda/base/envs/gp/lib/python3.13/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-03 to the diagonal
  warnings.warn(


### Plot GP Solution

In [53]:
# model.eval()
# likelihood.eval()
# from plotting_utils import *

# for mmsi in models:
#     model = models[mmsi]
#     likelihood = likelihoods[mmsi]
#     # Get the corresponding data for this MMSI
#     # If you want to use the same train/test split as before:
#     times, state_trajectory = None, None
#     for entry in gp_regression_dataset:
#         if entry[0] == mmsi:
#             _, times, state_trajectory = entry
#             break
#     if times is None:
#         continue  # skip if MMSI not found

#     train_X = times.clone().detach().unsqueeze(1).cpu()
#     train_Y = state_trajectory.clone().detach().cpu()

#     test_X = torch.linspace(times.min(), times.max(), 500).unsqueeze(1).to(device)
    
#     test_Y = eval_model(model, likelihood, test_X)

#     plot_gp(train_X, train_Y, test_X, test_Y)
#     plot_single_ship_path(mmsi, times, state_trajectory)


In [54]:
# import ipywidgets as widgets
# from IPython.display import display, clear_output
# from plotting_utils import *


# def plot_for_mmsi(selected_mmsi):
#     clear_output(wait=True)
#     model = models[selected_mmsi]
#     likelihood = likelihoods[selected_mmsi]
#     # Get the corresponding data for this MMSI
#     times, state_trajectory = None, None
#     for entry in gp_regression_dataset:
#         if entry[0] == selected_mmsi:
#             _, times, state_trajectory = entry
#             break
#     if times is None:
#         print("No data for MMSI:", selected_mmsi)
#         return

#     train_X = times.clone().detach().unsqueeze(1).cpu()
#     train_Y = state_trajectory.clone().detach().cpu()
#     test_X = torch.linspace(times.min(), times.max(), 500).unsqueeze(1).to(device)
#     test_Y = eval_model(model, likelihood, test_X)

#     plot_gp(train_X, train_Y, test_X, test_Y)
#     plot_single_ship_path(selected_mmsi, times, state_trajectory)

    
# mmsi_dropdown = widgets.Dropdown(
#     options=list(models.keys()),
#     description='MMSI:',
#     disabled=False,
# )

# widgets.interact(plot_for_mmsi, selected_mmsi=mmsi_dropdown)
    
    
    

In [55]:
print(pd.unique(gp_regression_dataset.df['MMSI'].values))
gp_regression_dataset.get_vessel_group_by_mmsi(3660489)

[367669550 367118980 636018568 ... 367619000 309108000 368926390]


'Other'

## Create the kernel param to ship mmsi dataset


In [95]:
kernel_classification_dataset = GPKernelShipClassificationDataset(gp_regression_dataset, models, device)
unique_group_ids = kernel_classification_dataset.get_unique_group_ids()

train_classification_dataset, test_classification_dataset = torch.utils.data.random_split(kernel_classification_dataset, [0.8, 0.2])
train_loader = torch.utils.data.DataLoader(train_classification_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_classification_dataset, batch_size=32, shuffle=False)

## Train the classification model

In [100]:
from gp_kernel_ship_classification_network import GPKernelShipClassificationNetwork
model = GPKernelShipClassificationNetwork(input_dim=2, num_classes=max(unique_group_ids) + 1)  # +1 for background class

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

trainer = GPKernelShipClassificationTrainer(model, train_loader, test_loader, criterion, optimizer, device)
trainer.train(num_epochs=500)

Epoch 500/500 | Train Loss: 0.6812 | Train Acc: 0.7500 | Test Loss: 4.4817 | Test Acc: 0.4000
